In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-269483
Azure region: westeurope
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-269483


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "test-alll"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

     
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.1, 4.0),
    '--max_iter': choice(50, 100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=6)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('./', script='train.py', compute_target=cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

2024-11-04 16:51:21.354016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 16:51:22.819227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 16:51:23.261814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 16:51:26.493672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-04 16:51:33.303704: W tensorflow/comp

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_37a2ed6b-29a0-48a4-81d7-8aad24639662
Web View: https://ml.azure.com/runs/HD_37a2ed6b-29a0-48a4-81d7-8aad24639662?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-269483/workspaces/quick-starts-ws-269483&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-04T16:50:39.5391695Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-11-04T16:50:40.0180343Z][SCHEDULER][INFO]Scheduling job, id='HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0' 
[2024-11-04T16:50:40.1049765Z][SCHEDULER][INFO]Scheduling job, id='HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_1' 
[2024-11-04T16:50:40.2232029Z][SCHEDULER][INFO]Scheduling job, id='HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_2' 
[2024-11-04T16:50:40.2219792Z][SCHEDULER][INFO]Scheduling job, id='HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_3' 
[2024-11-04T16:50:41.1130325Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_3' 
[2024-11-04T16:5

{'runId': 'HD_37a2ed6b-29a0-48a4-81d7-8aad24639662',
 'target': 'test-alll',
 'status': 'Completed',
 'startTimeUtc': '2024-11-04T16:50:37.919225Z',
 'endTimeUtc': '2024-11-04T17:08:51.02397Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cc13cbf3-ed4a-46ec-8879-1a071cb9a8dc',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0',
  'score': '0.9099645928174',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',

In [25]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
print(best_run)

Run(Experiment: udacity-project,
Id: HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

print(best_run_parameter_values)

Best Run Id:  HD_37a2ed6b-29a0-48a4-81d7-8aad24639662_0
Accuracy: 0.9099645928174
['--C', '1.6996979340435514', '--max_iter', '300']


In [14]:
# joblib.dump(value=best_run, filename='./model.joblib')

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [17]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [18]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()

Submitting remote run.
No run_configuration provided, running on test-alll with default configuration
Running on remote compute: test-alll


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5f5a62c9-548e-4a53-8ba5-8f0fc7713c01,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [19]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run_job, best_automl_model = automl_run.get_output()
joblib.dump(value=best_automl_model, filename='automl_model.joblib')

['automl_model.joblib']

In [ ]:
cluster.delete()